In [34]:
# Importing necessary libraries 
import pandas as pd
import geopandas as gpd
import psycopg2
from shapely.geometry import Point, Polygon
import pandas as pd
from sqlalchemy import create_engine

In [35]:
# QA\QC check
# Read the CSV file into a DataFrame
csv_file_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\rounded_gdd_data.csv"
df = pd.read_csv(csv_file_path)

# Check for null values
null_values = df.isnull().sum()
if null_values.any():
    print("Null values found:")
    print(null_values)
else:
    print("No null values found.")


Null values found:
Station             0
Date                0
Max Temp (F)      277
Min Temp (F)      277
Latitude            0
Longitude           0
GDD               277
Cumulative_GDD    277
dtype: int64


In [36]:
# Define Minnesota boundary box
minnesota_boundary = Polygon([( -97.5, 43.0), (-89.0, 43.0), (-89.0, 49.5), (-97.5, 49.5)])

# Check if the data falls within the Minnesota boundary
df['Coordinates'] = list(zip(df.Longitude, df.Latitude))
df['Coordinates'] = df['Coordinates'].apply(Point)
gdf = gpd.GeoDataFrame(df, geometry='Coordinates')

within_minnesota = gdf[gdf.geometry.within(minnesota_boundary)]
if len(within_minnesota) == 0:
    print("No data falls within Minnesota boundary.")
else:
    print("Data falls within Minnesota boundary.")

Data falls within Minnesota boundary.


In [60]:
# Upload the values from the csv to the table I have created in the database

# Read the CSV file into a DataFrame
csv_file_path = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\rounded_gdd_data.csv"
df = pd.read_csv(csv_file_path)


In [61]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

# Convert the datetime format to the desired format
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Now, the "Date" column is converted to dates in the format YYYY-MM-DD
print(df)

                                           Station  ... Cumulative_GDD
0                      Twin Lakes I-35 Mile Post 1  ...      20.339997
1                     Silver Lake TH 7 Mile Post 1  ...      43.469997
2                 Little Chicago I-35 Mile Post 70  ...      65.879986
3                     Rush City I-35 Mile Post 157  ...      88.739970
4                      Rutledge I-35 Mile Post 198  ...     111.419971
...                                            ...  ...            ...
4855  U.S.2 - Solway - MP 99.7         MN US MNDOT  ...   67633.560970
4856  T.H.1 - Warren - MP 11.5         MN US MNDOT  ...   67647.600970
4857  T.H.11 - Karlstad - MP 37.7      MN US MNDOT  ...   67661.100970
4858  T.H.371 - Nisswa - MP 46.2       MN US MNDOT  ...   67675.590960
4859  U.S.53 - Piedmont Ave - MP 2.7   MN US MNDOT  ...   67687.920960

[4860 rows x 8 columns]


In [64]:
#Now let's choose a date to interpolate for
input_date = input("Enter a date (YYYY-MM-DD format): ")

# Convert input string to datetime object
input_date = pd.to_datetime(input_date)

# Filter DataFrame based on the input date
df = df[df['Date'] == input_date]

# Display the filtered DataFrame
print(df)

Enter a date (YYYY-MM-DD format): 2023-09-01
                                          Station  ... Cumulative_GDD
0                     Twin Lakes I-35 Mile Post 1  ...      20.339997
1                    Silver Lake TH 7 Mile Post 1  ...      43.469997
2                Little Chicago I-35 Mile Post 70  ...      65.879986
3                    Rush City I-35 Mile Post 157  ...      88.739970
4                     Rutledge I-35 Mile Post 198  ...     111.419971
..                                            ...  ...            ...
157  U.S.2 - Solway - MP 99.7         MN US MNDOT  ...    3607.559940
158  T.H.1 - Warren - MP 11.5         MN US MNDOT  ...    3631.319929
159  T.H.11 - Karlstad - MP 37.7      MN US MNDOT  ...    3655.709929
160  T.H.371 - Nisswa - MP 46.2       MN US MNDOT  ...    3679.739930
161  U.S.53 - Piedmont Ave - MP 2.7   MN US MNDOT  ...    3703.139920

[162 rows x 8 columns]


In [65]:
# Define the output feature class name
output_fc = 'GDDPoints'

# Create a new feature class
arcpy.management.CreateFeatureclass(
    arcpy.env.workspace,
    output_fc,
    'POINT',
    spatial_reference=arcpy.SpatialReference(4326)  # WGS84 Geographic Coordinate System
)

# Add fields to store the necessary data, ensuring correct data types
arcpy.management.AddField(output_fc, 'Station', 'TEXT')
arcpy.management.AddField(output_fc, 'Date', 'DATE')
arcpy.management.AddField(output_fc, 'MaxTemp', 'FLOAT')
arcpy.management.AddField(output_fc, 'MinTemp', 'FLOAT')
arcpy.management.AddField(output_fc, 'Latitude', 'DOUBLE')
arcpy.management.AddField(output_fc, 'Longitude', 'DOUBLE')
arcpy.management.AddField(output_fc, 'GDD', 'FLOAT')
arcpy.management.AddField(output_fc, 'Cumulative_GDD', 'FLOAT')

# Open an insert cursor
with arcpy.da.InsertCursor(output_fc, ['SHAPE@XY', 'Station', 'Date', 'MaxTemp', 'MinTemp', 'Latitude', 'Longitude', 'GDD', 'Cumulative_GDD']) as cursor:
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Extract necessary values
        station = row['Station']
        date = row['Date']
        max_temp = row['Max Temp (F)']
        min_temp = row['Min Temp (F)']
        lat = row['Latitude']
        lon = row['Longitude']
        gdd = row['GDD']
        cumulative_gdd = row['Cumulative_GDD']
        
        # Create a point geometry
        point = arcpy.Point(lon, lat)
        point_geometry = arcpy.PointGeometry(point)
        
        # Insert the point feature with the necessary values
        cursor.insertRow([point_geometry, station, date, max_temp, min_temp, lat, lon, gdd, cumulative_gdd])

print(f"Feature class '{output_fc}' created successfully.")

Feature class 'GDDPoints' created successfully.


In [66]:
# IDW
arcpy.ga.IDW(
    in_features="GDDPoints",
    z_field="GDD",#From Owen: changed this from MaxTemp
    out_ga_layer=None,
    out_raster=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\GDD_IDW",
    cell_size=0.0218500824,
    power=2,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=2.32298810990939 S_MINOR=2.32298810990939 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    weight_field=None
)

<Result ''>

In [67]:
# ordinary kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="GDDPoints",
        z_field="GDD",#From Owen: changed this from MaxTemp
        kriging_model="Spherical # # # #",
        cell_size=0.0218500824,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Kriging") 


In [68]:
# Universal Kriging
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="GDDPoints",
        z_field="GDD",#From Owen: changed this from MaxTemp
        kriging_model="LinearDrift 0.021850 # # #",
        cell_size=0.0218500824,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Kriging_univ")  



In [ ]:
# QC
# exploratory Interpolation
arcpy.ga.ExploratoryInterpolation(
    in_features="GDDPoints",
    value_field="GDD",
    out_cv_table=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\PostgreSQL-34-final_project(postgres).sde\ExploratoryInterpolation1",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)



In [69]:
#Ths resamples the raster
arcpy.management.Resample(
    in_raster=r"Kriging_univ",
    out_raster=r"Kriging_univ_resample",
    cell_size="0.1 0.1",
    resampling_type="NEAREST"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Kriging_univ_resample'>

In [70]:
# And this turns the raster to points
arcpy.conversion.RasterToPoint(
    in_raster=r"Kriging_univ_resample",
    out_point_features=r"Kriging_univ_point",
    raster_field="value"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5572 Final\\GIS 5572 Final.gdb\\Kriging_univ_point'>

In [1]:
# This will export the Kriging points to a PostGIS database
arcpy.conversion.ExportFeatures(
    in_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\GIS 5572 Final.gdb\Kriging_univ_point",
    out_features=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5572 Final\PostgreSQL-34-final_project(postgres).sde\final_project.postgres.gdd_data"
)
print("Export complete")

Export complete
